In [1]:
'''
Graph and Loss visualization using Tensorboard.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
import pickle
from dataset_def import Dataset



In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 10000
batch_size = 128
display_step = 10
class_num = 5
logs_path = '/tmp/tensorflow_logs/example'

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, class_num], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, class_num]), name='Weights')
b = tf.Variable(tf.zeros([class_num]), name='Bias')

# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
cost_summ = tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
acc_summ = tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge([cost_summ, acc_summ]);



In [3]:
# Load data
tf.logging.info('loading data...')
with open('data/shapes/2D_train_dataset.pickle', 'rb') as data:
    train_dataset = pickle.load(data)
    
with open('data/shapes/2D_test_dataset.pickle', 'rb') as data:
    test_dataset = pickle.load(data)
tf.logging.info('DONE')

INFO:tensorflow:loading data...
INFO:tensorflow:DONE


In [5]:
num_examples = 1000
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = train_dataset.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            test_batch_x, test_batch_y = test_dataset.next_batch(256)
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.4f}".format(avg_cost),"accuracy=", "{:.4f}".format(acc.eval({x: test_batch_x, y: test_batch_y})))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    test_batch_x, test_batch_y = test_dataset.next_batch(256)
    print("Accuracy:", acc.eval({x: test_batch_x, y: test_batch_y}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0010 cost= 0.5497 accuracy= 0.6500
Epoch: 0020 cost= 0.5091 accuracy= 0.6900
Epoch: 0030 cost= 0.4988 accuracy= 0.7100
Epoch: 0040 cost= 0.4876 accuracy= 0.7100
Epoch: 0050 cost= 0.4645 accuracy= 0.7100
Epoch: 0060 cost= 0.4741 accuracy= 0.7200
Epoch: 0070 cost= 0.4551 accuracy= 0.7200
Epoch: 0080 cost= 0.4558 accuracy= 0.7100
Epoch: 0090 cost= 0.4559 accuracy= 0.7100
Epoch: 0100 cost= 0.4556 accuracy= 0.7100
Epoch: 0110 cost= 0.4391 accuracy= 0.7100
Epoch: 0120 cost= 0.4453 accuracy= 0.7100
Epoch: 0130 cost= 0.4591 accuracy= 0.7100
Epoch: 0140 cost= 0.4221 accuracy= 0.7100
Epoch: 0150 cost= 0.4241 accuracy= 0.7100
Epoch: 0160 cost= 0.4339 accuracy= 0.7200
Epoch: 0170 cost= 0.4533 accuracy= 0.7200
Epoch: 0180 cost= 0.3981 accuracy= 0.7200
Epoch: 0190 cost= 0.4211 accuracy= 0.7200
Epoch: 0200 cost= 0.4214 accuracy= 0.7300
Epoch: 0210 cost= 0.4292 accuracy= 0.7300
Epoch: 0220 cost= 0.4311 accuracy= 0.7300
Epoch: 0230 cost= 0.4058 accuracy= 0.7300
Epoch: 0240 cost= 0.4083 accuracy=

Epoch: 1970 cost= 0.2971 accuracy= 0.7500
Epoch: 1980 cost= 0.3035 accuracy= 0.7500
Epoch: 1990 cost= 0.3006 accuracy= 0.7500
Epoch: 2000 cost= 0.3086 accuracy= 0.7500
Epoch: 2010 cost= 0.2947 accuracy= 0.7500
Epoch: 2020 cost= 0.2977 accuracy= 0.7600
Epoch: 2030 cost= 0.2955 accuracy= 0.7500
Epoch: 2040 cost= 0.3009 accuracy= 0.7500
Epoch: 2050 cost= 0.3037 accuracy= 0.7500
Epoch: 2060 cost= 0.2932 accuracy= 0.7600
Epoch: 2070 cost= 0.2905 accuracy= 0.7500
Epoch: 2080 cost= 0.3082 accuracy= 0.7600
Epoch: 2090 cost= 0.2963 accuracy= 0.7600
Epoch: 2100 cost= 0.2930 accuracy= 0.7600
Epoch: 2110 cost= 0.3214 accuracy= 0.7600
Epoch: 2120 cost= 0.2977 accuracy= 0.7500
Epoch: 2130 cost= 0.2922 accuracy= 0.7500
Epoch: 2140 cost= 0.2897 accuracy= 0.7600
Epoch: 2150 cost= 0.3062 accuracy= 0.7600
Epoch: 2160 cost= 0.2951 accuracy= 0.7600
Epoch: 2170 cost= 0.2988 accuracy= 0.7500
Epoch: 2180 cost= 0.2900 accuracy= 0.7500
Epoch: 2190 cost= 0.2891 accuracy= 0.7600
Epoch: 2200 cost= 0.2911 accuracy=

Epoch: 3950 cost= 0.2547 accuracy= 0.7700
Epoch: 3960 cost= 0.2594 accuracy= 0.7600
Epoch: 3970 cost= 0.2442 accuracy= 0.7700
Epoch: 3980 cost= 0.2523 accuracy= 0.7700
Epoch: 3990 cost= 0.2574 accuracy= 0.7600
Epoch: 4000 cost= 0.2500 accuracy= 0.7700
Epoch: 4010 cost= 0.2579 accuracy= 0.7600
Epoch: 4020 cost= 0.2476 accuracy= 0.7600
Epoch: 4030 cost= 0.2509 accuracy= 0.7700
Epoch: 4040 cost= 0.2602 accuracy= 0.7600
Epoch: 4050 cost= 0.2533 accuracy= 0.7700
Epoch: 4060 cost= 0.2515 accuracy= 0.7600
Epoch: 4070 cost= 0.2496 accuracy= 0.7600
Epoch: 4080 cost= 0.2410 accuracy= 0.7600
Epoch: 4090 cost= 0.2628 accuracy= 0.7600
Epoch: 4100 cost= 0.2485 accuracy= 0.7700
Epoch: 4110 cost= 0.2412 accuracy= 0.7700
Epoch: 4120 cost= 0.2545 accuracy= 0.7700
Epoch: 4130 cost= 0.2426 accuracy= 0.7700
Epoch: 4140 cost= 0.2429 accuracy= 0.7600
Epoch: 4150 cost= 0.2461 accuracy= 0.7600
Epoch: 4160 cost= 0.2614 accuracy= 0.7600
Epoch: 4170 cost= 0.2491 accuracy= 0.7700
Epoch: 4180 cost= 0.2451 accuracy=

Epoch: 5910 cost= 0.2332 accuracy= 0.8000
Epoch: 5920 cost= 0.2177 accuracy= 0.8100
Epoch: 5930 cost= 0.2104 accuracy= 0.7900
Epoch: 5940 cost= 0.2239 accuracy= 0.8000
Epoch: 5950 cost= 0.2174 accuracy= 0.8000
Epoch: 5960 cost= 0.2306 accuracy= 0.8100
Epoch: 5970 cost= 0.2340 accuracy= 0.8100
Epoch: 5980 cost= 0.2142 accuracy= 0.8100
Epoch: 5990 cost= 0.2194 accuracy= 0.8100
Epoch: 6000 cost= 0.2183 accuracy= 0.8100
Epoch: 6010 cost= 0.2263 accuracy= 0.8100
Epoch: 6020 cost= 0.2264 accuracy= 0.8100
Epoch: 6030 cost= 0.2239 accuracy= 0.8100
Epoch: 6040 cost= 0.2140 accuracy= 0.8100
Epoch: 6050 cost= 0.2152 accuracy= 0.8000
Epoch: 6060 cost= 0.2128 accuracy= 0.8100
Epoch: 6070 cost= 0.2192 accuracy= 0.8000
Epoch: 6080 cost= 0.2155 accuracy= 0.8100
Epoch: 6090 cost= 0.2114 accuracy= 0.8100
Epoch: 6100 cost= 0.2109 accuracy= 0.8100
Epoch: 6110 cost= 0.2128 accuracy= 0.8100
Epoch: 6120 cost= 0.2179 accuracy= 0.8000
Epoch: 6130 cost= 0.2253 accuracy= 0.8100
Epoch: 6140 cost= 0.2209 accuracy=

Epoch: 7890 cost= 0.1929 accuracy= 0.8000
Epoch: 7900 cost= 0.2043 accuracy= 0.8000
Epoch: 7910 cost= 0.1948 accuracy= 0.8000
Epoch: 7920 cost= 0.2065 accuracy= 0.8000
Epoch: 7930 cost= 0.1938 accuracy= 0.8000
Epoch: 7940 cost= 0.1975 accuracy= 0.8000
Epoch: 7950 cost= 0.1901 accuracy= 0.8000
Epoch: 7960 cost= 0.2015 accuracy= 0.8000
Epoch: 7970 cost= 0.2011 accuracy= 0.8000
Epoch: 7980 cost= 0.1982 accuracy= 0.8000
Epoch: 7990 cost= 0.2001 accuracy= 0.8000
Epoch: 8000 cost= 0.1947 accuracy= 0.8000
Epoch: 8010 cost= 0.1995 accuracy= 0.8000
Epoch: 8020 cost= 0.1949 accuracy= 0.8000
Epoch: 8030 cost= 0.1968 accuracy= 0.8000
Epoch: 8040 cost= 0.1968 accuracy= 0.8000
Epoch: 8050 cost= 0.1906 accuracy= 0.8000
Epoch: 8060 cost= 0.2038 accuracy= 0.8000
Epoch: 8070 cost= 0.1929 accuracy= 0.8000
Epoch: 8080 cost= 0.1922 accuracy= 0.8000
Epoch: 8090 cost= 0.1888 accuracy= 0.8000
Epoch: 8100 cost= 0.1958 accuracy= 0.8000
Epoch: 8110 cost= 0.1879 accuracy= 0.8000
Epoch: 8120 cost= 0.1942 accuracy=

Epoch: 9870 cost= 0.1798 accuracy= 0.8000
Epoch: 9880 cost= 0.1809 accuracy= 0.8000
Epoch: 9890 cost= 0.1779 accuracy= 0.8000
Epoch: 9900 cost= 0.1770 accuracy= 0.8000
Epoch: 9910 cost= 0.1822 accuracy= 0.8000
Epoch: 9920 cost= 0.1740 accuracy= 0.8000
Epoch: 9930 cost= 0.1902 accuracy= 0.8000
Epoch: 9940 cost= 0.1814 accuracy= 0.8000
Epoch: 9950 cost= 0.1858 accuracy= 0.8000
Epoch: 9960 cost= 0.1823 accuracy= 0.8000
Epoch: 9970 cost= 0.1848 accuracy= 0.8000
Epoch: 9980 cost= 0.1749 accuracy= 0.8000
Epoch: 9990 cost= 0.1842 accuracy= 0.8000
Epoch: 10000 cost= 0.1815 accuracy= 0.8000
Optimization Finished!
Accuracy: 0.8
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
